In [56]:
from django.conf import settings
import geopandas
import pandas as pd
#from django_plotly_dash import DjangoDash
from sqlalchemy import create_engine
from dash import dcc,dash_table,html,Dash
from dash.dependencies import Input,Output,State
import dash_bootstrap_components as dbc
import plotly.express as px
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import shapely
from django_plotly_dash import DjangoDash

HOST ='localhost' #settings.DATABASES['default']['HOST']
NAME= 'elecciones23'#settings.DATABASES['default']['NAME']
PASS='4rcg1s2024'#settings.DATABASES['default']['PASSWORD']
conn = create_engine("postgresql+psycopg2://elecciones23:"+PASS+"@"+HOST+":5432/"+NAME)

In [57]:
sql="""
 SELECT * FROM public.despliegue_repliegue
 """
df = geopandas.GeoDataFrame.from_postgis(sql, conn)
df.to_crs(epsg=4326, inplace=True)
df['repliegue']=round(df['replegado']*100/df['total'],2)
distritos=df.drop_duplicates('distrito')['distrito'].sort_values()
def estado_urnas(distrito):
    if distrito and distrito != 'TODOS':
        sql_mesas="""
        SELECT * FROM public.mesas_estados where distrito='"""+distrito+"""'"""
    else:
        sql_mesas="""
        SELECT * FROM public.mesas_estados """   
    df_mesas= pd.read_sql(sql_mesas, conn)
    df_mesas=df_mesas.drop(columns=['punto', ])
    urnas=len(df_mesas)-df_mesas['entrego_correo'].sum()
    urnas_en_correo=df_mesas['entrego_correo'].sum()-df_mesas['entrego_urna_en_led'].sum()

    urnas_en_led=df_mesas['entrego_urna_en_led'].sum()
    return [str(urnas),str(urnas_en_correo),str(urnas_en_led)]

sql_estado_local="""select *
from cantidad_votos_local where estado_local='NO FINALIZADO' """
df_local= geopandas.GeoDataFrame.from_postgis(sql_estado_local, conn, geom_col='ubicacion').to_crs(4326)
df_local['lon'] = df_local['ubicacion'].x
df_local['lat'] = df_local['ubicacion'].y

def crear_mapa(datos_mapa, zoom, lat_cen=None, lon_cen=None):
    if lat_cen and lon_cen:
        center = dict(lat=lat_cen, lon=lon_cen)
    else:
        center = dict()
    mapa = px.scatter_mapbox(data_frame=datos_mapa, lat="lat", lon="lon",
                             hover_name="nombre",
                             color='validado',
                             color_continuous_scale=[[0, 'red'], [0.5, 'orange'], [1.0, 'red']],
                             range_color=[0, 1],
                           
                             hover_data=dict(distrito=True,
                                             subdistrito=True,
                                             nombre=True,
                                             seccion=True,
                                             circuito=True,
                                             direccion=True,
                                             grado_jefe_local=True,
                                             nombre_jefe_local=True,
                                             apellido_jefe_local=True,
                                             telefono_jefe_local=True,
                                             castidad_auxiliares=True,
                                             cant_efectivos=True,
                                             estado_causa=True,
                                             lat=False,
                                             lon=False,
                                             ),
                              zoom=zoom, height=750, center=center,
                              
                             )
    mapa.update_traces(

        marker={'size': 13},
        hovertemplate="<br>".join(["<b>Nombre:</b> %{customdata[2]}",
                                                  "<b>Distrito:</b> %{customdata[0]}",
                                                  "<b>Subdistrito:</b> %{customdata[1]}",
                                                  "<b>Sección:</b> %{customdata[3]}",
                                                  "<b>Circuito:</b> %{customdata[4]}",
                                                  "<b>Dirección:</b> %{customdata[5]}",
                                                  "<b>J loc:</b> %{customdata[6]} %{customdata[8]} %{customdata[7]}",
                                                  "<b>Tel J Loc:</b> %{customdata[9]}",
                                                  "<b>Auxiliares:</b> %{customdata[10]}",
                                                  "<b>Seg Ext:</b> %{customdata[11]}",
                                                    "<b>Estado Causa:</b> %{customdata[12]}",

                                                  ]), )
    mapa.update_layout(hoverlabel_bgcolor='#ffffff',coloraxis_showscale=False,

                       mapbox_style="open-street-map", margin={"r": 0, "t": 0, "l": 0, "b": 0}
                       )
    mapa.update_geos(fitbounds='locations')
    return mapa

crear_mapa(df_local, zoom=4)

In [58]:
def crear_porcentaje(df):
    data=round(df['repliegue'].mean(skipna = True),2)
    titulo='ESTADO DEL REPLIEGUE'
    
    fig = go.Figure(
        go.Indicator(
            mode='gauge+number',
            value=data,
            number={"font": {"size": 50,"color":'white'},'suffix': "%"},
            domain = {'x': [0, 1], 'y': [0, 1]},
            gauge = {
                'axis': {'range': [None, 100], 'tickwidth': 2,'tickcolor':'white',},
                'bar': {'color': "red"},
                'borderwidth': 2,
                'steps': [
                    {'range': [0, 50], 'color': 'pink'},
                    {'range': [50, 75], 'color': 'yellow'},
                    {'range': [75 ,100], 'color': 'green'}],
            }
        )
    )

    fig.update_layout(height=700,title_text=titulo,autosize=True, 
        font={'color': "white", 'family': "Arial",'size':20},grid={'rows': 1, 'columns': 1, 'pattern': "independent"}, title_x=0.5,
        
        plot_bgcolor='rgb(40, 40, 40)',
    
        paper_bgcolor='rgb(40, 40, 40)',
    )
    return fig

In [59]:
def crear_tabla(df_mov_lista):
    df_mov_lista = df_mov_lista.drop(columns=['ubicacion', ])
    data_table = dash_table.DataTable(df_mov_lista.to_dict('records'),
                                    [{"name": i, "id": i.lower()} for i in ['Nombre']],
                                    id='datatable',
                                    editable=False,
                                    
                                    sort_action="native",
                                    sort_mode="multi",
                                    selected_columns=[],
                                    selected_rows=[],
                                
                                    page_action="native",
                                    page_current=0,
                                    page_size=13,
                                    style_cell={'textAlign': 'center','fontSize':12, 'font-family':'sans-serif'},
                                    style_data={
                                        'color': 'black',
                                        'backgroundColor': 'white'

                                    },
                                    style_header={
                                        'backgroundColor': 'rgb(210, 210, 210)',
                                        'color': 'black',
                                        'fontWeight': 'bold'
                                    }

                                    )
    return data_table
crear_tabla(df_local)

DataTable(data=[{'id': 21985, 'distrito': 'CHACO', 'subdistrito': '-', 'seccion': '24', 'circuito': '130', 'nombre': 'E.G.B. N° 240 "PEDRO ARATA"', 'localidad': 'C.DEL BERMEJO', 'direccion': '9 DE JULIO 14', 'validado': 1, 'cantidad_mesas': 8, 'cant_efectivos': 2, 'jefe_local_id': 101178, 'fuerza_jefe_local': 'EJERCITO', 'grado_jefe_local': '(SP) SUBOFICIAL PRINCIPAL', 'nombre_jefe_local': 'ALCIDES RAMON', 'apellido_jefe_local': 'LOPEZ', 'telefono_jefe_local': '3624005138', 'castidad_auxiliares': 1, 'votos': 65.0, 'hora': datetime.time(17, 30), 'estado_local': 'NO FINALIZADO', 'estado_causa': 'CONTINÚA VOTACIÓN', 'mesas_no_iniciada': None, 'no_subsanadas': None, 'cant_nov': None, 'situacion': 'Inicio repliegue', 'lon': -60.94509279730119, 'lat': -26.60118183176818}, {'id': 21986, 'distrito': 'CHACO', 'subdistrito': '-', 'seccion': '24', 'circuito': '131', 'nombre': 'E.G.B. N° 657 "DR.ANCHORENA"', 'localidad': 'PPA.DEL INFIERNO', 'direccion': 'ALFREDO BRUGNOLI Y RUTA 16', 'validado': 1,

In [60]:
app = Dash( external_stylesheets=[dbc.themes.DARKLY])
todos = [{"label": 'TODOS', "value": 'TODOS'}]
app.layout = html.Div(
        [   
            dbc.Row([html.H1('ESTADO DE LAS URNAS', style={'textAlign': 'center', 'background-color': '#267300'})]),
            dbc.Row([dbc.Select(id="selec_distrito",
                                options=todos + [{"label": x, "value": x} for x in distritos],placeholder='DISTRITO' )]),
            dbc.Row([dbc.Col([html.H3('URNAS EN LOCAL', style={'textAlign': 'center', 'background-color': '#ff0000'})]),
                     dbc.Col([html.H3(id='urnas', children=[estado_urnas('TODOS')[0]])]),
                     dbc.Col([html.H3('EN CORREO', style={'textAlign': 'center', 'background-color': '#e6e600'})]),
                     dbc.Col([html.H3(id='urnas_correo', children=[estado_urnas('TODOS')[1]])]),
                     dbc.Col([html.H3('URNAS EN LED', style={'textAlign': 'center', 'background-color': '#267300'})]),
                     dbc.Col([html.H3(id='urnas_led', children=[estado_urnas('TODOS')[2]])])
                                                
        ]),                    
            dbc.Row([dbc.Col([dbc.Row([dcc.Loading(id="ls-loading-2", children=[crear_tabla(df_local)], type="default")]),
                              
                              
                              ], style={"height": "100%", }
                             , md=4),
                     dbc.Col([dbc.Row([dcc.Loading(id="ls-loading-3", children=[dcc.Graph(id='mapa', figure=crear_mapa(df_local,4))], type="default")])], style={"height": "100%", }, md=5),

                     dbc.Col([dbc.Row([dcc.Loading(id="ls-loading-4", children=[dcc.Graph(id='total',figure=crear_porcentaje(df))], type="default")]),
                              ],style={"height": "100%", },md=3),
                     ], className="g-0", ),
             dcc.Interval(
                id='interval-component',
                interval=60 * 1000,  # in milliseconds
                n_intervals=0
            )
        ]
    )
app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [29/May/2025 12:07:11] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2025 12:07:11] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2025 12:07:11] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2025 12:07:11] "GET /_favicon.ico?v=2.9.3 HTTP/1.1" 200 -
127.0.0.1 - - [29/May/2025 12:07:11] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 304 -
127.0.0.1 - - [29/May/2025 12:07:11] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 304 -
127.0.0.1 - - [29/May/2025 12:07:11] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [29/May/2025 12:07:11] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
